In [14]:
import asyncio
import requests
from bs4 import BeautifulSoup

from playwright.async_api import async_playwright

In [15]:
root_path = "https://perm.hse.ru/education/"

bach = "bachelor"
ms = "magister"

In [17]:
res = requests.get(root_path+bach)

In [18]:
soap = BeautifulSoup(res.content, "html.parser")

In [19]:
programs = {}
program_path = "div.edu-programm__group div.b-row"

data = soap.select(program_path)

for div in data:
    style = div.get("style", "")
    if "display: none" not in style:
        item = div.find("a", {"class": "link"})

        programs[item.text] = item.get("href", "") + "ratings"

In [20]:
programs

{'Дизайн': 'https://perm.hse.ru/ba/design/ratings',
 'Программные системы и автоматизация процессов разработки': 'https://perm.hse.ru/ba/ssadp/ratings',
 'Разработка информационных систем для бизнеса': 'https://perm.hse.ru/ba/isystems/ratings',
 'Международный бакалавриат по бизнесу и экономике': 'https://perm.hse.ru/ba/interbac/ratings',
 'Юриспруденция': 'https://perm.hse.ru/ba/law/ratings',
 'Иностранные языки и межкультурная коммуникация в бизнесе': 'https://perm.hse.ru/ba/communication/ratings',
 'Бизнес-аналитика': 'https://perm.hse.ru/ma/iaem/ratings',
 'Государственное и муниципальное управление': 'https://perm.hse.ru/ma/gmu/ratings',
 'Управление развитием бизнеса': 'https://perm.hse.ru/ma/bdm/ratings',
 'Финансовые стратегии и аналитика': 'https://perm.hse.ru/ma/fs/ratings',
 'ИТ-юрист': 'https://perm.hse.ru/ma/ltlawyer/ratings',
 'Правовое обеспечение предпринимательской деятельности': 'https://perm.hse.ru/ma/ls/ratings'}

In [13]:
def get_all_ratings(url: str):
    ...

def get_all_courses(url: str):
    ...

# def parse_rating_table(url: str):
    

#     response = requests.get(url)

#     if response.status_code != 200:
#         return

#     soap = BeautifulSoup(response.content, "html.parser")

#     selector = "tr.o"
#     data = soap.select(selector)

#     return data

async def parse_rating_table(url: str):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)  # Запускаем браузер в headless режиме
        page = await browser.new_page()

        await page.goto(url)

        await page.wait_for_selector("tr.o", timeout=10000)  # Ожидаем, что элементы с классом 'o' появятся

        html = await page.content()

        soup = BeautifulSoup(html, 'html.parser')

        data = soup.select("tr.o")

        await browser.close()

        return data

async def main():
    data = await parse_rating_table(programs["Дизайн"])
    print(data)


if __name__ == "__main__":
    asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop